<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

## Introduction

## Data Cleaning

### Initial Wrangling

In [83]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_columns = None
import sys
import geopy
from uszipcode import SearchEngine, SimpleZipcode, Zipcode
import numpy as np

In [2]:
cols_to_remove = ['Business - Dogs Allowed', 'Business - Address', 'Business - Smoking', 'Business - Restaurant?', 'Business - Outdoor Seating', 'Business - Order at Counter', 'Business - Happy Hour', 'Business - Good For Kids', 'Business - Good For Dancing', 'Business - Corkage', 'Business - Coat Check', 'Business - BYOB/Corkage', 'Business - BYOB', 'Business - Monday Close','Business - Saturday Close','Business - Friday Open','Business - Saturday Open','Business - Monday Open','Business - Sunday Close','Business - Sunday Open','Business - Wednesday Close','Business - Thursday Close','Business - Wednesday Open','Business - Thursday Open','Business - Tuesday Close','Business - Tuesday Open','Business - Friday Close','Number of Records','Business - Accepts Insurance','Business - Parking','User - Name','Business - Neighborhoods','Business - Open?','Business - State','User - Compliments Cool','User - Compliments Cute','User - Compliments Funny','User - Compliments Hot','User - Compliments List','User - Compliments More','User - Compliments Note','User - Compliments Photos','User - Compliments Plain','User - Compliments Profile','User - Compliments Writer','User - Votes Cool','User - Votes Funny','User - Votes Useful']

In [3]:
df = pd.read_csv('./Data/yelp_data.csv')
df = df[df.columns.difference(cols_to_remove)]

In [4]:
display(df.head())
print(df.shape)

,Business - Accepts Credit Cards,Business - Ages Allowed,Business - Alcohol,Business - Attire,Business - By Appointment Only,Business - Categories,Business - Caters,Business - City,Business - Delivery,Business - Dietary Restrictions,Business - Drive-Thru,Business - Good For Groups,Business - Good for Kids,Business - Has TV,Business - Id,Business - Name,Business - Noise Level,Business - Open 24 Hours,Business - Payment Types,Business - Price Range,Business - Review Count,Business - Stars,Business - Take-out,Business - Takes Reservations,Business - Waiter Service,Business - Wheelchair Accessible,Business - Wi-Fi,Latitude,Longitude,Review - Date,Review - Id,Review - Stars,Review - Votes Cool,Review - Votes Funny,Review - Votes Useful,User - Average Stars,User - Fans,User - Id,User - Review Count,User - Years Elite,User - Yelping Since
0,True,NaN,beer_and_wine,casual,NaN,"Gluten-Free,Asian Fusion,Chinese,Restaurants",False,Gilbert,False,NaN,NaN,True,True,False,ghzksdgNztTZA49PqMvPkA,Pei Wei Asian Diner,loud,NaN,NaN,1.0,36,3.5,True,False,False,True,no,33.330011,-111.790909,2/1/2005,xSaWTOB24fK69MDNcjyuOg,4,0,0,0,4.00,0,gKhJyiCKkG6dHPNhr3dhDQ,1,NaN,2004-10
1,True,NaN,full_bar,casual,NaN,"Chinese,Restaurants",True,Chandler,False,NaN,NaN,True,True,False,hrOsdyai-9B91cGjBspHqw,Tao Garden,average,NaN,NaN,2.0,55,4.0,True,False,True,True,no,33.335842,-111.859086,3/7/2005,I6Snobdz2kX_sqmzi-1K5A,3,0,0,0,3.80,4,kJc9YBRwmmZ_PG0uLHuEPQ,192,NaN,2005-02
2,True,NaN,beer_and_wine,casual,NaN,"Italian,Pizza,Sandwiches,Restaurants",False,Phoenix,False,NaN,NaN,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,False,NaN,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01
3,True,NaN,full_bar,dressy,NaN,"Steakhouses,Restaurants",False,Phoenix,False,NaN,NaN,True,False,True,cN6aBxe2mQvrQlzk26LyRQ,Durant's,average,False,NaN,3.0,418,4.0,False,True,True,True,no,33.477236,-112.073483,3/29/2005,KpOZEomlHF_tgJJXttQTwA,5,2,1,3,4.38,8,6VZNGc2h2Bn-uyuEXgOt5g,130,2005 2006 2007,2005-03
4,True,NaN,full_bar,casual,NaN,"Chinese,Restaurants",True,Phoenix,True,NaN,NaN,True,True,False,a8yzGF79KcfyRye2R6wt3Q,Golden Phoenix Chinese Restaurant,average,NaN,NaN,2.0,48,3.5,True,True,True,True,no,33.524850,-112.047889,4/7/2005,D9Dc32OAIpHUI3p7OFBWMQ,4,1,0,2,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01


(285764, 41)


Total restaurants

In [5]:
df['Business - Id'].unique().shape

(5696,)

Examine value counts and NAs and remove features with low variance or too many NAs.

In [6]:
print(df['Business - Ages Allowed'].isna().sum())
df['Business - Ages Allowed'].value_counts()

285285


21plus     431
allages     29
18plus      19
Name: Business - Ages Allowed, dtype: int64

In [7]:
print(df['Business - By Appointment Only'].isna().sum())
df['Business - By Appointment Only'].value_counts()

285217


False    538
True       9
Name: Business - By Appointment Only, dtype: int64

In [8]:
print(df['Business - Dietary Restrictions'].isna().sum())

279849


In [9]:
print(df['Business - Drive-Thru'].isna().sum())
df['Business - Drive-Thru'].value_counts()

260833


False    17798
True      7133
Name: Business - Drive-Thru, dtype: int64

In [10]:
print(df['Business - Open 24 Hours'].isna().sum())
df['Business - Open 24 Hours'].value_counts()

267126


False    18594
True        44
Name: Business - Open 24 Hours, dtype: int64

In [11]:
print(df['Business - Payment Types'].isna().sum())
df['Business - Payment Types'].value_counts()

283638


{u'amex': True, u'cash_only': False, u'mastercard': True, u'visa': True, u'discover': True}       1020
{}                                                                                                 793
{u'amex': False, u'cash_only': False, u'visa': True, u'mastercard': True, u'discover': False}      135
{u'amex': True, u'cash_only': False, u'visa': True, u'mastercard': True, u'discover': False}       101
{u'amex': False, u'cash_only': True, u'visa': False, u'mastercard': False, u'discover': False}      77
Name: Business - Payment Types, dtype: int64

In [12]:
cols_to_remove = ['Business - Payment Types', 'Business - Open 24 Hours', 'Business - Drive-Thru', 'Business - Dietary Restrictions', 'Business - Ages Allowed', 'Business - By Appointment Only', 'Business - Dogs Allowed', 'Business - Address', 'Business - Smoking', 'Business - Restaurant?', 'Business - Outdoor Seating', 'Business - Order at Counter', 'Business - Happy Hour', 'Business - Good For Kids', 'Business - Good For Dancing', 'Business - Corkage', 'Business - Coat Check', 'Business - BYOB/Corkage', 'Business - BYOB', 'Business - Monday Close','Business - Saturday Close','Business - Friday Open','Business - Saturday Open','Business - Monday Open','Business - Sunday Close','Business - Sunday Open','Business - Wednesday Close','Business - Thursday Close','Business - Wednesday Open','Business - Thursday Open','Business - Tuesday Close','Business - Tuesday Open','Business - Friday Close','Number of Records','Business - Accepts Insurance','Business - Parking','User - Name','Business - Neighborhoods','Business - Open?','Business - State','User - Compliments Cool','User - Compliments Cute','User - Compliments Funny','User - Compliments Hot','User - Compliments List','User - Compliments More','User - Compliments Note','User - Compliments Photos','User - Compliments Plain','User - Compliments Profile','User - Compliments Writer','User - Votes Cool','User - Votes Funny','User - Votes Useful']

In [13]:
df = df[df.columns.difference(cols_to_remove)]
display(df.head())
print(df.shape)

,Business - Accepts Credit Cards,Business - Alcohol,Business - Attire,Business - Categories,Business - Caters,Business - City,Business - Delivery,Business - Good For Groups,Business - Good for Kids,Business - Has TV,Business - Id,Business - Name,Business - Noise Level,Business - Price Range,Business - Review Count,Business - Stars,Business - Take-out,Business - Takes Reservations,Business - Waiter Service,Business - Wheelchair Accessible,Business - Wi-Fi,Latitude,Longitude,Review - Date,Review - Id,Review - Stars,Review - Votes Cool,Review - Votes Funny,Review - Votes Useful,User - Average Stars,User - Fans,User - Id,User - Review Count,User - Years Elite,User - Yelping Since
0,True,beer_and_wine,casual,"Gluten-Free,Asian Fusion,Chinese,Restaurants",False,Gilbert,False,True,True,False,ghzksdgNztTZA49PqMvPkA,Pei Wei Asian Diner,loud,1.0,36,3.5,True,False,False,True,no,33.330011,-111.790909,2/1/2005,xSaWTOB24fK69MDNcjyuOg,4,0,0,0,4.00,0,gKhJyiCKkG6dHPNhr3dhDQ,1,NaN,2004-10
1,True,full_bar,casual,"Chinese,Restaurants",True,Chandler,False,True,True,False,hrOsdyai-9B91cGjBspHqw,Tao Garden,average,2.0,55,4.0,True,False,True,True,no,33.335842,-111.859086,3/7/2005,I6Snobdz2kX_sqmzi-1K5A,3,0,0,0,3.80,4,kJc9YBRwmmZ_PG0uLHuEPQ,192,NaN,2005-02
2,True,beer_and_wine,casual,"Italian,Pizza,Sandwiches,Restaurants",False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01
3,True,full_bar,dressy,"Steakhouses,Restaurants",False,Phoenix,False,True,False,True,cN6aBxe2mQvrQlzk26LyRQ,Durant's,average,3.0,418,4.0,False,True,True,True,no,33.477236,-112.073483,3/29/2005,KpOZEomlHF_tgJJXttQTwA,5,2,1,3,4.38,8,6VZNGc2h2Bn-uyuEXgOt5g,130,2005 2006 2007,2005-03
4,True,full_bar,casual,"Chinese,Restaurants",True,Phoenix,True,True,True,False,a8yzGF79KcfyRye2R6wt3Q,Golden Phoenix Chinese Restaurant,average,2.0,48,3.5,True,True,True,True,no,33.524850,-112.047889,4/7/2005,D9Dc32OAIpHUI3p7OFBWMQ,4,1,0,2,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01


(285764, 35)


Recode and examine reviews by years.

In [14]:
df['ReviewYear'] = pd.DatetimeIndex(df['Review - Date']).year

In [15]:
display(df['ReviewYear'].value_counts())

2013    75071
2014    58946
2012    49378
2011    43506
2010    28689
2009    17055
2008     9566
2007     2896
2006      608
2005       49
Name: ReviewYear, dtype: int64

### Clean Business Categories

Explode cateogories

In [16]:
df['Business - Categories'] = df['Business - Categories'].str.split(",")

In [17]:
dfExp = df.explode('Business - Categories').copy()

Display top 20 categories for resutaurants

In [18]:
display(pd.DataFrame(dfExp['Business - Categories'].value_counts().iloc[1:]).head(20))

,Business - Categories
Nightlife,43428
American (New),43392
Bars,41424
Mexican,38665
American (Traditional),34386
Pizza,32477
Italian,30288
Breakfast & Brunch,27211
Sandwiches,27043
Food,24814


In [19]:
restVals = pd.DataFrame(dfExp['Business - Categories'].value_counts().iloc[1:].head(20))

Remove redundant categories

In [20]:
topList = restVals.index.tolist().copy()
topList.remove('Nightlife')
topList.remove('Food')
topList.remove('Pubs')
topList.remove('Sports Bars')
topList.remove('Bars')
print(topList)

['American (New)', 'Mexican', 'American (Traditional)', 'Pizza', 'Italian', 'Breakfast & Brunch', 'Sandwiches', 'Burgers', 'Sushi Bars', 'Chinese', 'Steakhouses', 'Japanese', 'Seafood', 'Mediterranean', 'Fast Food']


In [21]:
topCuisines = dfExp[dfExp['Business - Categories'].isin(topList)]

In [38]:
del dfExp

In [22]:
topCuisines['Business - Id'].unique().shape

(4818,)

In [23]:
print(topCuisines.shape)
topCuisines.head()

(336048, 36)


,Business - Accepts Credit Cards,Business - Alcohol,Business - Attire,Business - Categories,Business - Caters,Business - City,Business - Delivery,Business - Good For Groups,Business - Good for Kids,Business - Has TV,Business - Id,Business - Name,Business - Noise Level,Business - Price Range,Business - Review Count,Business - Stars,Business - Take-out,Business - Takes Reservations,Business - Waiter Service,Business - Wheelchair Accessible,Business - Wi-Fi,Latitude,Longitude,Review - Date,Review - Id,Review - Stars,Review - Votes Cool,Review - Votes Funny,Review - Votes Useful,User - Average Stars,User - Fans,User - Id,User - Review Count,User - Years Elite,User - Yelping Since,ReviewYear
0,True,beer_and_wine,casual,Chinese,False,Gilbert,False,True,True,False,ghzksdgNztTZA49PqMvPkA,Pei Wei Asian Diner,loud,1.0,36,3.5,True,False,False,True,no,33.330011,-111.790909,2/1/2005,xSaWTOB24fK69MDNcjyuOg,4,0,0,0,4.00,0,gKhJyiCKkG6dHPNhr3dhDQ,1,NaN,2004-10,2005
1,True,full_bar,casual,Chinese,True,Chandler,False,True,True,False,hrOsdyai-9B91cGjBspHqw,Tao Garden,average,2.0,55,4.0,True,False,True,True,no,33.335842,-111.859086,3/7/2005,I6Snobdz2kX_sqmzi-1K5A,3,0,0,0,3.80,4,kJc9YBRwmmZ_PG0uLHuEPQ,192,NaN,2005-02,2005
2,True,beer_and_wine,casual,Italian,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005
2,True,beer_and_wine,casual,Pizza,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005
2,True,beer_and_wine,casual,Sandwiches,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005


In [37]:
df = topCuisines.copy()

In [56]:
del topCuisines

### Clean Cities

In [39]:
df.loc[df['Business - City'] == 'Phoenix Sky Harbor Center', ['Business - City']] = 'Phoenix'
df.loc[df['Business - City'] == 'Pheonix', ['Business - City']] = 'Phoenix'

In [42]:
topCities = df['Business - City'].value_counts().head(10).index.tolist()

In [44]:
topCities = df.loc[df['Business - City'].isin(topCities)].copy()

In [45]:
print(topCities.shape)
topCities.head()

(320097, 36)


,Business - Accepts Credit Cards,Business - Alcohol,Business - Attire,Business - Categories,Business - Caters,Business - City,Business - Delivery,Business - Good For Groups,Business - Good for Kids,Business - Has TV,Business - Id,Business - Name,Business - Noise Level,Business - Price Range,Business - Review Count,Business - Stars,Business - Take-out,Business - Takes Reservations,Business - Waiter Service,Business - Wheelchair Accessible,Business - Wi-Fi,Latitude,Longitude,Review - Date,Review - Id,Review - Stars,Review - Votes Cool,Review - Votes Funny,Review - Votes Useful,User - Average Stars,User - Fans,User - Id,User - Review Count,User - Years Elite,User - Yelping Since,ReviewYear
0,True,beer_and_wine,casual,Chinese,False,Gilbert,False,True,True,False,ghzksdgNztTZA49PqMvPkA,Pei Wei Asian Diner,loud,1.0,36,3.5,True,False,False,True,no,33.330011,-111.790909,2/1/2005,xSaWTOB24fK69MDNcjyuOg,4,0,0,0,4.00,0,gKhJyiCKkG6dHPNhr3dhDQ,1,NaN,2004-10,2005
1,True,full_bar,casual,Chinese,True,Chandler,False,True,True,False,hrOsdyai-9B91cGjBspHqw,Tao Garden,average,2.0,55,4.0,True,False,True,True,no,33.335842,-111.859086,3/7/2005,I6Snobdz2kX_sqmzi-1K5A,3,0,0,0,3.80,4,kJc9YBRwmmZ_PG0uLHuEPQ,192,NaN,2005-02,2005
2,True,beer_and_wine,casual,Italian,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005
2,True,beer_and_wine,casual,Pizza,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005
2,True,beer_and_wine,casual,Sandwiches,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005


In [57]:
df = topCities.copy()
del topCities

### Assign Zip Codes

In [85]:
zipmap = df.drop_duplicates(subset=['Latitude','Longitude'])[['Latitude','Longitude']]

In [88]:
zipmap

,Latitude,Longitude
0,33.330011,-111.790909
1,33.335842,-111.859086
2,33.449233,-112.065458
3,33.477236,-112.073483
4,33.524850,-112.047889
...,...,...
279922,33.421531,-111.947576
280615,33.499192,-111.917816
280892,33.474235,-112.074605
281929,33.480030,-112.080959


In [89]:
def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    print(location.raw['address']['postcode'])
    return location.raw['address']['postcode']

In [90]:
zipcodes1 = zipmap.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='Latitude', lon_field='Longitude')
zipcodes

85269
85225
85004
85004-1118
85016-1739
85013
85210-4913
85260-1235
85021-3516
85012-1711
85006
85260-6568
85254
85257
85254
85253
85251
85259
85258
85251
85251
85251
85281
85251
85251
85287
85023
85018
85018-8830
8587
85287
85281
85202
8525
85048
85281
85021
85021
85308
85257
85224
85260
85260-1235
85253
85251
85254
85012-1711
85006
85018
85224
85260-8516
85260-1235
85260-8516
85254
85250
85253
85260-1235
85215
85257
85042
85251
85255
85253
85251
85287
85018
85257
85251
85251
85004
85281
85251
85251
85018
85251
85016-1739
85260-1235
85003


KeyboardInterrupt: 

### Exploratory Data Analysis

In [ ]:
df['Business - Alcohol']

In [79]:
search

In [93]:
df

,Business - Accepts Credit Cards,Business - Alcohol,Business - Attire,Business - Categories,Business - Caters,Business - City,Business - Delivery,Business - Good For Groups,Business - Good for Kids,Business - Has TV,Business - Id,Business - Name,Business - Noise Level,Business - Price Range,Business - Review Count,Business - Stars,Business - Take-out,Business - Takes Reservations,Business - Waiter Service,Business - Wheelchair Accessible,Business - Wi-Fi,Latitude,Longitude,Review - Date,Review - Id,Review - Stars,Review - Votes Cool,Review - Votes Funny,Review - Votes Useful,User - Average Stars,User - Fans,User - Id,User - Review Count,User - Years Elite,User - Yelping Since,ReviewYear
0,True,beer_and_wine,casual,Chinese,False,Gilbert,False,True,True,False,ghzksdgNztTZA49PqMvPkA,Pei Wei Asian Diner,loud,1.0,36,3.5,True,False,False,True,no,33.330011,-111.790909,2/1/2005,xSaWTOB24fK69MDNcjyuOg,4,0,0,0,4.00,0,gKhJyiCKkG6dHPNhr3dhDQ,1,NaN,2004-10,2005
1,True,full_bar,casual,Chinese,True,Chandler,False,True,True,False,hrOsdyai-9B91cGjBspHqw,Tao Garden,average,2.0,55,4.0,True,False,True,True,no,33.335842,-111.859086,3/7/2005,I6Snobdz2kX_sqmzi-1K5A,3,0,0,0,3.80,4,kJc9YBRwmmZ_PG0uLHuEPQ,192,NaN,2005-02,2005
2,True,beer_and_wine,casual,Italian,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005
2,True,beer_and_wine,casual,Pizza,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005
2,True,beer_and_wine,casual,Sandwiches,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285760,True,full_bar,casual,Breakfast & Brunch,True,Tempe,False,True,True,True,0K634ehk41ZRc4kj3NTAEQ,Chompie's Deli,average,2.0,316,3.5,True,True,True,True,no,33.422413,-111.919903,7/16/2014,mOVR0cOK_kjMmFgUC6jCzQ,5,0,0,0,3.67,1,BZ4OhVIIS1LSAVvnb3q9fg,13,NaN,2013-11,2014
285762,True,full_bar,casual,Mediterranean,True,Phoenix,False,True,False,False,EWMwV5V9BxNs_U6nNVMeqw,FEZ,average,2.0,1046,4.0,True,True,True,True,free,33.491695,-112.073498,7/16/2014,2-1DaV_uHkw5UDOx02-cqw,3,0,0,0,2.62,0,xueMNSdcJuP_C-jhUygpLA,21,NaN,2009-04,2014
285762,True,full_bar,casual,American (New),True,Phoenix,False,True,False,False,EWMwV5V9BxNs_U6nNVMeqw,FEZ,average,2.0,1046,4.0,True,True,True,True,free,33.491695,-112.073498,7/16/2014,2-1DaV_uHkw5UDOx02-cqw,3,0,0,0,2.62,0,xueMNSdcJuP_C-jhUygpLA,21,NaN,2009-04,2014
285763,True,full_bar,casual,Mediterranean,True,Phoenix,False,True,False,False,EWMwV5V9BxNs_U6nNVMeqw,FEZ,average,2.0,1046,4.0,True,True,True,True,free,33.491695,-112.073498,7/16/2014,-4wywkiDE-lcHrak7VYM9A,4,0,0,0,3.40,0,7XcwzFPtcTHHbd1_dEOdKw,5,NaN,2014-04,2014


In [95]:
df

,Business - Accepts Credit Cards,Business - Alcohol,Business - Attire,Business - Categories,Business - Caters,Business - City,Business - Delivery,Business - Good For Groups,Business - Good for Kids,Business - Has TV,Business - Id,Business - Name,Business - Noise Level,Business - Price Range,Business - Review Count,Business - Stars,Business - Take-out,Business - Takes Reservations,Business - Waiter Service,Business - Wheelchair Accessible,Business - Wi-Fi,Latitude,Longitude,Review - Date,Review - Id,Review - Stars,Review - Votes Cool,Review - Votes Funny,Review - Votes Useful,User - Average Stars,User - Fans,User - Id,User - Review Count,User - Years Elite,User - Yelping Since,ReviewYear
0,True,beer_and_wine,casual,Chinese,False,Gilbert,False,True,True,False,ghzksdgNztTZA49PqMvPkA,Pei Wei Asian Diner,loud,1.0,36,3.5,True,False,False,True,no,33.330011,-111.790909,2/1/2005,xSaWTOB24fK69MDNcjyuOg,4,0,0,0,4.00,0,gKhJyiCKkG6dHPNhr3dhDQ,1,NaN,2004-10,2005
1,True,full_bar,casual,Chinese,True,Chandler,False,True,True,False,hrOsdyai-9B91cGjBspHqw,Tao Garden,average,2.0,55,4.0,True,False,True,True,no,33.335842,-111.859086,3/7/2005,I6Snobdz2kX_sqmzi-1K5A,3,0,0,0,3.80,4,kJc9YBRwmmZ_PG0uLHuEPQ,192,NaN,2005-02,2005
2,True,beer_and_wine,casual,Italian,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005
2,True,beer_and_wine,casual,Pizza,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005
2,True,beer_and_wine,casual,Sandwiches,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285760,True,full_bar,casual,Breakfast & Brunch,True,Tempe,False,True,True,True,0K634ehk41ZRc4kj3NTAEQ,Chompie's Deli,average,2.0,316,3.5,True,True,True,True,no,33.422413,-111.919903,7/16/2014,mOVR0cOK_kjMmFgUC6jCzQ,5,0,0,0,3.67,1,BZ4OhVIIS1LSAVvnb3q9fg,13,NaN,2013-11,2014
285762,True,full_bar,casual,Mediterranean,True,Phoenix,False,True,False,False,EWMwV5V9BxNs_U6nNVMeqw,FEZ,average,2.0,1046,4.0,True,True,True,True,free,33.491695,-112.073498,7/16/2014,2-1DaV_uHkw5UDOx02-cqw,3,0,0,0,2.62,0,xueMNSdcJuP_C-jhUygpLA,21,NaN,2009-04,2014
285762,True,full_bar,casual,American (New),True,Phoenix,False,True,False,False,EWMwV5V9BxNs_U6nNVMeqw,FEZ,average,2.0,1046,4.0,True,True,True,True,free,33.491695,-112.073498,7/16/2014,2-1DaV_uHkw5UDOx02-cqw,3,0,0,0,2.62,0,xueMNSdcJuP_C-jhUygpLA,21,NaN,2009-04,2014
285763,True,full_bar,casual,Mediterranean,True,Phoenix,False,True,False,False,EWMwV5V9BxNs_U6nNVMeqw,FEZ,average,2.0,1046,4.0,True,True,True,True,free,33.491695,-112.073498,7/16/2014,-4wywkiDE-lcHrak7VYM9A,4,0,0,0,3.40,0,7XcwzFPtcTHHbd1_dEOdKw,5,NaN,2014-04,2014


In [94]:
df.to_csv("cleanDF.csv")

### Report

In [ ]:
Overall, I found